In [1]:
import numpy as np
%pip install scipy --upgrade
import math
import matplotlib.pyplot as plt
import scipy.stats as stats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.


In [2]:
#Simulate demand data
np.random.seed(9527)

n=5000
temp=np.random.uniform(0,30,n)
weather=np.random.binomial(1,0.3,n)

demand=np.zeros(n)
for i in range(n):
  mu_d=5*temp[i]+50
  if weather[i]==0: sigma_d=0.5*mu_d
  else: sigma_d=0.6*mu_d
  d=np.random.normal(mu_d,sigma_d)
  if d<0: d=0
  demand[i]=d

print(temp[0:10])
print(weather[0:10])
print(demand[0:10])

[ 9.92044955 13.43312395  7.04903863 28.44601981 23.76884188 27.80970292
 17.71132662 10.99108418  8.0699376  25.93554339]
[1 0 0 0 0 1 0 0 1 0]
[149.27771845 147.68114425 140.1597962  150.18420846 324.72786791
 120.42596578 311.49141948  47.18988438 129.78639333 199.83862194]


In [3]:
p=8000
c=2000
s=0
Cu=p-c
Co=c-s

sum_profit=0
sum_Q=0
for i in range(n):
  mu_d=5*temp[i]+50
  Q=mu_d
  d=demand[i]
  profit=-c*Q
  if d<=Q: profit=profit+p*d+s*(Q-d)
  else: profit=profit+p*Q
  sum_Q=sum_Q+Q
  sum_profit=sum_profit+profit

avg_Q=sum_Q/n
avg_profit=sum_profit/n
print("Quantity=",avg_Q)
print("Expected profit=",avg_profit)

Quantity= 124.571692270642
Expected profit= 540994.6992436528


In [4]:
import tensorflow as tf
def inventory_loss(demand_true, q):
  stock=q-demand_true
  is_understock=stock<0
  inventory_error=tf.abs(stock)
  understock_loss=Cu*inventory_error
  overstock_loss=Co*inventory_error
  return tf.where(is_understock, understock_loss, overstock_loss)

In [5]:
x_train=np.zeros((n,2))
x_train[:,0]=(temp-min(temp))/(max(temp)-min(temp))
x_train[:,1]=weather
demand_train=(demand-min(demand))/(max(demand)-min(demand))

In [6]:
from tensorflow import keras
model=keras.Sequential([
    keras.layers.Dense(2,input_shape=(2,),activation='sigmoid'),
    keras.layers.Dense(1,activation=None),
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer='adam',loss=inventory_loss)
model.fit(x_train,demand_train,epochs=20)

Epoch 1/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 754.0297
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 562.8675
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 472.8244
Epoch 4/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 417.9902
Epoch 5/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 377.6353
Epoch 6/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 349.5010
Epoch 7/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 342.2074
Epoch 8/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 330.6565
Epoch 9/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 320.9987
Epoch 10/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 327.0235
Epoch 11/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 325.4329
Epoch 12/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 322.3495
Epoch 13/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 325.6596
Epoch 14/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 327.8972
Epoch 15/20
157/157 ━━━━━━━━━

In [8]:
n_test=5000
temp_new=np.random.uniform(0,30,n_test)
weather_new=np.random.binomial(1,0.3,n_test)
x_test=np.zeros((n_test,2))
x_test[:,0]=(temp_new-min(temp))/(max(temp)-min(temp))
x_test[:,1]=weather_new

pred=model.predict(x_test)
stats.describe(pred)

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


DescribeResult(nobs=5000, minmax=(array([0.11462304], dtype=float32), array([0.5136452], dtype=float32)), mean=array([0.31941867], dtype=float32), variance=array([0.01258622], dtype=float32), skewness=array([-0.10091436], dtype=float32), kurtosis=array([-1.2058223], dtype=float32))

In [9]:
Q_hat=pred*(max(demand)-min(demand))+min(demand)
print("Mean of Q_hat=",np.mean(Q_hat))
stats.describe(Q_hat)

Mean of Q_hat= 167.64456


DescribeResult(nobs=5000, minmax=(array([60.15907], dtype=float32), array([269.58292], dtype=float32)), mean=array([167.64456], dtype=float32), variance=array([3466.9973], dtype=float32), skewness=array([-0.10091443], dtype=float32), kurtosis=array([-1.205822], dtype=float32))

In [10]:
np.sum(Q_hat)/n

167.6445625

In [11]:
sum_profit=0
sum_Q=0
for i in range(n_test):
  mu_d=5*temp_new[i]+50
  Q=mu_d
  if weather_new[i]==0: sigma_d=0.5*mu_d
  else: sigma_d=0.6*mu_d
  d=np.random.normal(mu_d,sigma_d)
  if d<0: d=0
  #
  Q=Q_hat[i]
  profit=-c*Q
  if d<=Q: profit=profit+p*d+s*(Q-d)
  else: profit=profit+p*Q
  sum_Q=sum_Q+Q
  sum_profit=sum_profit+profit

avg_Q=sum_Q/n_test
avg_profit=sum_profit/n_test
print("Quantity=",avg_Q)
print("Expected profit=",avg_profit)

Quantity= [167.64455]
Expected profit= [587000.4]
